In [ ]:

# # Types: repeat, bounce, random
# ANIMATION_STATE_CONFIGS: dict[str, dict[str, dict]] = {
#     "test": {
#         "states": {
#             "idle": {
#                 "type": "repeat",
#                 "delay": 30,
#                 "frames": [0, 1],
#             },
#             "move": {
#                 "type": "bounce",
#                 "delay": 10,
#                 "frames": [0, 1, 2],
#             }
#         }
#     },
#     "bubble": {
#         "states": {
#             "idle": {
#                 "type": "bounce",
#                 "delay": 5,
#                 "frames": [0, 1, 2, 3, 4, 5, 6, 7, 8],
#             }
#         }
#     }
# }
# for key, info in ANIMATION_STATE_CONFIGS.items():
#     for state_name, state_info in info["states"].items():
#         if state_info["type"] == "bounce":
#             # The frames are inverted and trimmed:
#             # [0, 1, 2, 3] -> [3, 2, 1, 0] -> [_, 2, 1, _]
#             #   Becomes [0, 1, 2, 3, 2, 1] ... [0, 1, 2, 3, ...]
#             # Both first and last are removed to avoid dupe frames:
#             # [0, 1, 2, 3] + [3, 2, 1, 0] -> [0, 1, 2, 3, 3, 2, 1, 0, 0, 1, ...] -> 0 and 3 are duped on loop
#             tail = state_info["frames"][::-1][1:-1]
#             state_info["frames"] += tail
#         state_info["length"] = len(state_info["frames"])

# def parse_direc_frame(file_name: str) -> tuple[int, int]:
#     direc, frame = None, 0
    
#     for info in file_name.split('_'):
#         if not info[1:].isnumeric():
#             continue
#         val = int(info[1:])
        
#         if info.startswith('d'): direc = val
#         if info.startswith('f'): frame = val
    
#     return direc, frame


# BASE_PATH = "./rotsys/sprites/animations/"
# ANIM_PATHS: dict[str, dict[str, str]] = {}
# for dirpath, _, filenames in os.walk(BASE_PATH):
#     anim_name = dirpath.replace(BASE_PATH, "")
#     if not anim_name or not filenames:
#         continue
#     ANIM_PATHS[anim_name] = {fn.split('.')[0]: f"{dirpath}/{fn}" for fn in filenames}

# TEXTURES = {}
# for name, paths in ANIM_PATHS.items():
#     TEXTURES[name] = {}
#     for texname, texpath in paths.items():
#         key = parse_direc_frame(texname)
#         texture = arcade.load_texture(texpath)
#         TEXTURES[name][key] = texture


In [ ]:

class PlayerCharacter(arcade.Sprite):
    def __init__(self, frames, states, is_fixed: bool = False):
        super().__init__()
        self.current_state = "idle"
        self.current_direc = 0
        self.current_frame = 0
        self.frame_counter = 0
        
        self.states: dict[str, dict] = states
        self.frames: dict[tuple, any] = frames
        
        self.is_fixed = is_fixed # If True, does not rotate
        if is_fixed:
            self.current_direc = None
    
    def update_animation(self, delta_time: float = 1 / 60):
        # Defines state
        old_state = self.current_state
        
        if not self.is_fixed:
            if self.change_x or self.change_y: # Only changes orientation if there is change
                self.current_state = "move"
                self.current_direc = math.atan2(self.change_y, self.change_x) / math.pi * 180
            else:
                self.current_state = "idle"
        
        # Resets counters on state change
        if old_state != self.current_state:
            self.frame_counter = 0
            self.current_frame = 0
        
        state_info = self.states.get(self.current_state)
        
        # Defines a frame change
        self.frame_counter += 1
        if self.frame_counter >= state_info.get("delay"):
            self.frame_counter = 0
            
            match state_info.get("type"):
                case "repeat" | "bounce":
                    self.current_frame = (self.current_frame + 1) % state_info.get("length")
                case "random":
                    self.current_frame = random.randint(0, state_info.get("length")-1)
        
        if self.is_fixed:
            direc = self.current_direc
        else:
            direc = utils.stick(self.current_direc, [a for a in range(0, 360, 360//8)], 360)
        frame = state_info["frames"][self.current_frame]
        self.texture = self.frames.get((direc, frame))


In [ ]:
        # self.player = PlayerCharacter(
        #     TEXTURES["bubble"],
        #     ANIMATION_STATE_CONFIGS["bubble"]["states"],
        #     True,
        # )
        # self.player = PlayerCharacter(
        #     TEXTURES["test"],
        #     ANIMATION_STATE_CONFIGS["test"]["states"],
        #     False,
        # )

In [ ]:


class AnimatedSprite(arcade.Sprite):
    def __init__(self, animation_data: AnimationData, **kwargs) -> None:
        super().__init__()
        self.cur_state: str = None
        self.cur_angle: float = 0
        self.cur_direction: int = 0
        self.cur_frame_index: int = 0
        self.cur_frame_count: int = 0
        self.cur_frame_limit: int = 0
        
        self.change_d: float = 0
        
        self.cur_state_changed: bool = False
        self.cur_frame_changed: bool = False
        self.cur_direc_changed: bool = False
        
        self.animation: AnimationData = animation_data
        
        rots = self.animation.keyframes["configs"]["directions"]
        if rots is not None and rots >= 2:
            self.can_rotate = True
            self.directions: list[int] = [int(3600*a/rots) for a in range(rots)]
        
        # self.configs = kwargs
        self.update_animation()
    
    def _cur_state_info(self) -> dict:
        return self.animation.keyframes["states"][self.cur_state]
    
    def on_update(self, delta_time: float = 1 / 60):
        super().on_update(delta_time)
        self.cur_angle += self.change_d
    
    def update(self):
        super().update()
        self.cur_angle += self.change_d
    
    def update_state(self) -> None:
        # State logic can change drastically from model to model, must override
        last_state = self.cur_state
        
        # ... logic here ...
        if self.change_x or self.change_y:
            self.cur_state = "move"
        else:
            self.cur_state = "idle"
        
        self.cur_state_changed = last_state != self.cur_state
    
    def update_frame(self) -> None:
        self.cur_frame_changed = False
        state_info = self._cur_state_info()
        
        if self.cur_state_changed:
            self.cur_frame_index = 0
            self.cur_frame_count = 0
            self.cur_frame_changed = True
        
        frame_limit = state_info["delay"][self.cur_frame_index]
        
        if self.cur_frame_count >= frame_limit-1:
            self.cur_frame_index = (self.cur_frame_index + 1) % state_info["length"]
            self.cur_frame_count = -1
            self.cur_frame_changed = True
        
        self.cur_frame_count += 1
    
    def update_direc(self) -> None:
        if not self.can_rotate:
            return
        
        last_direc = self.cur_direction
        
        if self.cur_state in ("idle"):
            pass
            #self.cur_angle += 0.01
        else:
            self.cur_angle = math.atan2(self.change_y, self.change_x)
        
        direc = 10 * math.degrees(self.cur_angle)
        self.cur_direction = utils.stick(direc, self.directions, 3600)
        self.cur_direc_changed = last_direc != self.cur_direction
    
    def update_texture(self) -> None:
        direc = self.cur_direction
        frame = self._cur_state_info()["frames"][self.cur_frame_index]
        self.texture = self.animation.textures.get((direc, frame))
    
    def update_animation(self, delta_time: float = 1/60) -> None:
        self.update_state()
        self.update_frame()
        self.update_direc()
        
        if any([self.cur_state_changed, self.cur_frame_changed, self.cur_direc_changed]):
            self.update_texture()



In [ ]:
    # def on_mouse_motion(self, x: int, y: int, dx: int, dy: int):
    #     self.mouse_pointer.position = x, y
    
    # def on_mouse_press(self, x: int, y: int, button: int, modifiers: int):
    #     for s in self.general_sprites:
    #         match button:
    #             case arcade.MOUSE_BUTTON_LEFT:  s.lock_on(self.mouse_pointer)
    #             case arcade.MOUSE_BUTTON_RIGHT: s.lock_off()
    
    # def on_key_press(self, key: arcade.key, modifiers):
    #     p: AnimatedSprite
    #     for p in self.general_sprites:
    #         match key:
    #             case arcade.key.UP:    p.change_y = MOVEMENT_SPEED
    #             case arcade.key.DOWN:  p.change_y = -MOVEMENT_SPEED
    #             case arcade.key.LEFT:  p.change_x = -MOVEMENT_SPEED
    #             case arcade.key.RIGHT: p.change_x = MOVEMENT_SPEED
    #             case arcade.key.Q:     p.change_d = ROTATION_SPEED
    #             case arcade.key.E:     p.change_d = -ROTATION_SPEED
    #             case arcade.key.W:     p.scale    += 0.1
    #             case arcade.key.S:     p.scale    -= 0.1
    #             case arcade.key.SPACE: p.go_to(self.mouse_pointer.position, MOVEMENT_SPEED, 10)
    
    # def on_key_release(self, key: arcade.key, modifiers = None):
    #     p: AnimatedSprite
    #     for p in self.general_sprites:
    #         match key:
    #             case arcade.key.UP   | arcade.key.DOWN:  p.change_y = 0
    #             case arcade.key.LEFT | arcade.key.RIGHT: p.change_x = 0
    #             case arcade.key.Q    | arcade.key.E:     p.change_d = 0
    